In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 401.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=10
✅ 시뮬레이션 초기화 완료: S=10, Randomness='high', M1=680.00, M2=1045.00


In [2]:
m2 = gp.Model("set")
m2.setParam("MIPGap", 1e-6)
m2.setParam("Heuristics", 0.3)


a = m2.addVars(T, vtype=GRB.CONTINUOUS, name="alpha")
bp = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_plus")
bm = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_minus")
gc = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
eta = m2.addVars(T, S, vtype=GRB.BINARY, name="eta")
lam = m2.addVars(T, S, vtype=GRB.BINARY, name="lambda")
mu = m2.addVars(T, S, vtype=GRB.BINARY, name="mu")

x = m2.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = m2.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = m2.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = m2.addVars(I, T, S, vtype=GRB.BINARY, name="delta")
rho = m2.addVars(I, T, S, vtype=GRB.BINARY, name="rho")
req1 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="req")

dp = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dyp")
dm = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dym")
dcp = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc+")
dcm = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc-")
ddp = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd+")
ddm = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd-")

q1 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q1")
q2 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q2")
q3 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q3")
q4 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q4")
q5 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q5")
q6 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q6")
q7 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q7")

ep = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")
ec = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_charge")
ed = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_discharge")

m2.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum((1 / S) * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))

m2.setObjective(obj, GRB.MAXIMIZE)

# ---------individual-----------
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    m2.addConstr(yp[i, t, s] <= R[i, t, s])
    m2.addConstr(zd[i, t, s] <= z[i, t, s])
    m2.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    m2.addConstr(yp[i, t, s] <= M1 * rho[i, t, s])
    m2.addConstr(ym[i, t, s] <= M1 * (1 - rho[i, t, s]))
    m2.addConstr(ym[i, t, s] <= M1 * delta[i, t, s])
    m2.addConstr(zc[i, t, s] <= M1 * (1 - delta[i, t, s]))
    m2.addConstr(zc[i, t, s] <= M1 * zeta[i, t, s])
    m2.addConstr(zd[i, t, s] <= M1 * (1 - zeta[i, t, s]))
    m2.addConstr(z[i, t, s] <= K[i])
for i, s in product(range(I), range(S)):
    m2.addConstr(z[i, 0, s] == K0[i])

# -----------settle------------
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(dp[i, j, t, s] for j in range(I)))
    m2.addConstr(em[i, t, s] == ym[i, t, s] - dm[i, t, s])
    m2.addConstr(ec[i, t, s] == zc[i, t, s] - gp.quicksum(dcp[i, j, t, s] for j in range(I)) + dcm[i, t, s])
    m2.addConstr(ed[i, t, s] == zd[i, t, s] - ddp[i, t, s] + gp.quicksum(ddm[i, j, t, s] for j in range(I)))
    m2.addConstr(ec[i, t, s] <= K[i] - z[i, t, s])
    m2.addConstr(ed[i, t, s] <= z[i, t, s])
    m2.addConstr(z[i, t + 1, s] == z[i, t, s] + ec[i, t, s] - ed[i, t, s])
    
    m2.addConstr(gp.quicksum(dcp[i, j, t, s] for j in range(I)) <= M1 * q1[i, t, s])
    m2.addConstr(dcm[i, t, s] <= M1 * (1 - q1[i, t, s]))
    m2.addConstr(ddp[i, t, s] <= M1 * q2[i, t, s])
    m2.addConstr(gp.quicksum(ddm[i, j, t, s] for j in range(I)) <= M1 * (1 - q2[i, t, s]))
    m2.addConstr(ep[i, t, s] <= M1 * q3[i, t, s])
    m2.addConstr(em[i, t, s] <= M1 * (1 - q3[i, t, s]))
    m2.addConstr(em[i, t, s] <= M1 * q4[i, t, s])
    m2.addConstr(ec[i, t, s] <= M1 * (1 - q4[i, t, s]))
    m2.addConstr(ec[i, t, s] <= M1 * q5[i, t, s])
    m2.addConstr(ed[i, t, s] <= M1 * (1 - q5[i, t, s]))

#-----------settlement-----------
for t in range(T):
    m2.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
for t, s in product(range(T), range(S)):
    m2.addConstr(bp[t, s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    m2.addConstr(bm[t, s] == gp.quicksum(em[i, t, s] for i in range(I)))
    m2.addConstr(gc[t, s] == gp.quicksum(ec[i, t, s] for i in range(I)))
    m2.addConstr(gd[t, s] == gp.quicksum(ed[i, t, s] for i in range(I)))

#----------aggregation------------
for t, s in product(range(T), range(S)):
    m2.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s] + gc[t, s] - gd[t, s])
    m2.addConstr(bp[t, s] <= gp.quicksum(R[i, t, s] for i in range(I)))
    m2.addConstr(bp[t, s] <= M2 * mu[t, s])
    m2.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
    m2.addConstr(bm[t, s] <= M2 * eta[t, s])
    m2.addConstr(gc[t, s] <= M2 * (1 - eta[t, s]))
    
# ------logical sense---------
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(yp[i, t, s] >= gp.quicksum(dp[i, j, t, s] for j in range(I)))
    m2.addConstr(ym[i, t, s] >= dm[i, t, s])
    m2.addConstr(zc[i, t, s] >= gp.quicksum(dcp[i, j, t, s] for j in range(I)))
    m2.addConstr(zd[i, t, s] >= ddp[i, t, s])
    m2.addConstr(req1[i, t, s] <= ym[i, t, s])
    m2.addConstr(req1[i, t, s] <= z[i, t, s])
    m2.addConstr(req1[i, t, s] >= ym[i, t, s] - M1 * (1 - q6[i, t, s]))
    m2.addConstr(req1[i, t, s] >= z[i, t, s] - M1 * q6[i, t, s])
    m2.addConstr(zd[i, t, s] + ddm[i, i, t, s] >= req1[i, t, s])
    m2.addConstr(gp.quicksum(ym[i, t, s] for i in range(I)) >= gp.quicksum(dcp[i, j, t, s] for j in range(I)))
    m2.addConstr(gp.quicksum(ym[i, t, s] for i in range(I)) >= gp.quicksum(ddm[i, j, t, s] for j in range(I)))

# for i, j, t, s in product(range(I), range(I), range(T), range(S)):
#     m2.addConstr(gp.quicksum(dp[i, j, t, s] for j in range(I)) - gp.quicksum(dp[j, i, t, s] for j in range(I)) <= dp[i, i, t, s])
#     m2.addConstr(dp[i, i, t, s] >= dcm[i, t, s] + ddp[i, t, s]) 

# for i, t, s in product(range(I), range(T), range(S)):
#     for j in range(I):
#         if i != j:
#             m2.addConstr(dm[i, t, s] >= dp[j, i, t, s] + dcp[i, j, t, s] + ddm[i, j, t, s] + dcp[i, i, t, s] + ddm[i, i, t, s])
 
m2.optimize()

if m2.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {m2.objVal}")
else:
    print("No optimal solution found.")

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-09
Set parameter MIPGap to value 1e-06
Set parameter Heuristics to value 0.3
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  1e-06
Heuristics  0.3

Optimize a model with 30534 rows, 24846 columns and 87966 nonzeros
Model fingerprint: 0x983e39bd
Variable types: 16206 continuous, 8640 integer (8640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6292 rows and 8890 columns
Presolve time: 0.09s
Presolved: 24242 rows, 15956 columns, 65572 nonzeros
Variable types: 8986 continuous, 6970 integer (6970 binary)
Found heuristic solution: objective 4200

In [3]:
a_vals = np.array(
    [m2.getVarByName(f"alpha[{t}]").X for t in range(T)]
)
bp_vals = np.array(
    [[m2.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
bm_vals = np.array(
    [[m2.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
x_vals = np.array(
    [
        [[m2.getVarByName(f"x[{i},{t}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
yp_vals = np.array(
    [
        [
            [m2.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ym_vals = np.array(
    [
        [
            [m2.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zc_vals = np.array(
    [
        [
            [m2.getVarByName(f"z_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zd_vals = np.array(
    [
        [
            [m2.getVarByName(f"z_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
z_vals = np.array(
    [
        [[m2.getVarByName(f"z[{i},{t},{s}]").X for s in range(S)] for t in range(T + 1)]
        for i in range(I)
    ]
)
ep_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
em_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ec_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ed_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dp_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dyp[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
dm_vals = np.array(
    [
        [[m2.getVarByName(f"dym[{i},{t},{s}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
dcp_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzc+[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
dcm_vals = np.array(
    [
        [[m2.getVarByName(f"dzc-[{i},{t},{s}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
ddp_vals = np.array(
    [
        [[m2.getVarByName(f"dzd+[{i},{t},{s}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
ddm_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzd-[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)

In [ ]:
m2 = gp.Model("set")
m2.setParam("MIPGap", 1e-6)
m2.setParam("Heuristics", 0.3)


a = m2.addVars(T, vtype=GRB.CONTINUOUS, name="alpha")
bp = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_plus")
bm = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_minus")
gc = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = m2.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
eta = m2.addVars(T, S, vtype=GRB.BINARY, name="eta")s
lam = m2.addVars(T, S, vtype=GRB.BINARY, name="lambda")
mu = m2.addVars(T, S, vtype=GRB.BINARY, name="mu")

x = m2.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = m2.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = m2.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = m2.addVars(I, T, S, vtype=GRB.BINARY, name="delta")
rho = m2.addVars(I, T, S, vtype=GRB.BINARY, name="rho")
req1 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="req1")
req2 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="req2")

dp = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dyp")
dm = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dym")
dcp = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc+")
dcm = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc-")
ddp = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd+")
ddm = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd-")

q1 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q1")
q2 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q2")
q3 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q3")
q4 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q4")
q5 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q5")
q6 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q6")
q7 = m2.addVars(I, T, S, vtype=GRB.BINARY, name="q7")

ep = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")
ec = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_charge")
ed = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_discharge")

m2.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum((1 / S) * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))

m2.setObjective(obj, GRB.MAXIMIZE)

# ---------individual-----------
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    m2.addConstr(yp[i, t, s] <= R[i, t, s])
    m2.addConstr(zd[i, t, s] <= z[i, t, s])
    m2.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    m2.addConstr(yp[i, t, s] <= M1 * rho[i, t, s])
    m2.addConstr(ym[i, t, s] <= M1 * (1 - rho[i, t, s]))
    m2.addConstr(ym[i, t, s] <= M1 * delta[i, t, s])
    m2.addConstr(zc[i, t, s] <= M1 * (1 - delta[i, t, s]))
    m2.addConstr(zc[i, t, s] <= M1 * zeta[i, t, s])
    m2.addConstr(zd[i, t, s] <= M1 * (1 - zeta[i, t, s]))
    m2.addConstr(z[i, t, s] <= K[i])
for i, s in product(range(I), range(S)):
    m2.addConstr(z[i, 0, s] == K0[i])

# -----------settle------------
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(dp[i, j, t, s] for j in range(I)))
    m2.addConstr(em[i, t, s] == ym[i, t, s] - gp.quicksum(dm[i, j, t, s] for j in range(I)))
    m2.addConstr(ec[i, t, s] == zc[i, t, s] - gp.quicksum(dcp[i, j, t, s] for j in range(I)) + gp.quicksum(dcm[i, j, t, s] for j in range(I)))
    m2.addConstr(ed[i, t, s] == zd[i, t, s] - gp.quicksum(ddp[i, j, t, s] for j in range(I)) + gp.quicksum(ddm[i, j, t, s] for j in range(I)))
    m2.addConstr(ec[i, t, s] <= K[i] - z[i, t, s])
    m2.addConstr(ed[i, t, s] <= z[i, t, s])
    m2.addConstr(z[i, t + 1, s] == z[i, t, s] + ec[i, t, s] - ed[i, t, s])
    
    m2.addConstr(req1[i, t, s] <= ym[i, t, s])
    m2.addConstr(req1[i, t, s] <= z[i, t, s])
    m2.addConstr(req1[i, t, s] >= ym[i, t, s] - M1 * (1 - q6[i, t, s]))
    m2.addConstr(req1[i, t, s] >= z[i, t, s] - M1 * q6[i, t, s])
    m2.addConstr(ed[i, t, s] >= req1[i, t, s])
    
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(yp[i, t, s] >= gp.quicksum(dp[i, j, t, s] for j in range(I)))
    m2.addConstr(ym[i, t, s] >= gp.quicksum(dm[i, j, t, s] for j in range(I)))
    m2.addConstr(zc[i, t, s] >= gp.quicksum(dcp[i, j, t, s] for j in range(I)))
    m2.addConstr(zd[i, t, s] >= gp.quicksum(ddp[i, j, t, s] for j in range(I)))
    m2.addConstr(gp.quicksum(ym[i, t, s] for i in range(I)) >= gp.quicksum(dcp[i, j, t, s] for j in range(I)))
    m2.addConstr(gp.quicksum(ym[i, t, s] for i in range(I)) >= gp.quicksum(ddm[i, j, t, s] for j in range(I)))
    
for i, j, t, s in product(range(I), range(I), range(T), range(S)):
    m2.addConstr(gp.quicksum(dp[i, j, t, s] for j in range(I)) - gp.quicksum(dp[j, i, t, s] for j in range(I) if j != i) == dp[i, i, t, s])
    m2.addConstr(dp[i, i, t, s] >= dcm[i, i, t, s] + ddp[i, i, t, s]) #<=, ==가 맞는거같은데
    # m2.addConstr(dp[i, i, t, s] <= ec[i, t, s] + ddp[i, i, t, s])
    # if j != i:
    #     dcm[j, i, t, s] == 0
    #     ddp[j, i, t, s] == 0
    #     dcm[i, j, t, s] == 0
    #     ddp[i, j, t, s] == 0
    # m2.addConstr(req2[i, t, s] <= dp[i, i, t, s] - ddp[i, i, t, s] )
    # m2.addConstr(req2[i, t, s] <= K[i] - z[i, t, s])
    # m2.addConstr(req2[i, t, s] >= (dp[i, i, t, s] - ddp[i, i, t, s]) - M1 * (1 - q7[i, t, s]))
    # m2.addConstr(req2[i, t, s] >= (K[i] - z[i, t, s]) - M1 * q7[i, t, s])
    # m2.addConstr(ec[i, t, s] >= req2[i, t, s])
    m2.addConstr(req2[i, t, s] <= dp[i, i, t, s] - ddp[i, i, t, s] )
    m2.addConstr(req2[i, t, s] <= K[i] - z[i, t, s] - zc[i, t, s])
    m2.addConstr(req2[i, t, s] >= (dp[i, i, t, s] - ddp[i, i, t, s]) - M1 * (1 - q7[i, t, s]))
    m2.addConstr(req2[i, t, s] >= (K[i] - z[i, t, s] - zc[i, t, s]) - M1 * q7[i, t, s])
    m2.addConstr(dcm[i, i, t, s] >= req2[i, t, s])

for i, t, s in product(range(I), range(T), range(S)):
    for j in range(I):
#         # m2.addConstr(dm[i, i, t, s] >= dcp[i, i, t, s] + ddm[i, i, t, s]) ##곧 지울것. 고려하지 않기
        if j != i:
            m2.addConstr(dm[j, i, t, s] >= dp[j, i, t, s] + dcp[j, i, t, s] + ddm[j, i, t, s])
    
for i, t, s in product(range(I), range(T), range(S)):
    m2.addConstr(gp.quicksum(dcp[i, j, t, s] for j in range(I)) <= M1 * q1[i, t, s])
    m2.addConstr(dcm[i, i, t, s] <= M1 * (1 - q1[i, t, s]))
    m2.addConstr(ddp[i, i, t, s] <= M1 * q2[i, t, s])
    m2.addConstr(gp.quicksum(ddm[i, j, t, s] for j in range(I)) <= M1 * (1 - q2[i, t, s]))
    m2.addConstr(ep[i, t, s] <= M1 * q3[i, t, s])
    m2.addConstr(em[i, t, s] <= M1 * (1 - q3[i, t, s]))
    m2.addConstr(em[i, t, s] <= M1 * q4[i, t, s])
    m2.addConstr(ec[i, t, s] <= M1 * (1 - q4[i, t, s]))
    m2.addConstr(ec[i, t, s] <= M1 * q5[i, t, s])
    m2.addConstr(ed[i, t, s] <= M1 * (1 - q5[i, t, s]))

#-----------settlement-----------
for t in range(T):
    m2.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
for t, s in product(range(T), range(S)):
    m2.addConstr(bp[t, s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    m2.addConstr(bm[t, s] == gp.quicksum(em[i, t, s] for i in range(I)))
    m2.addConstr(gc[t, s] == gp.quicksum(ec[i, t, s] for i in range(I)))
    m2.addConstr(gd[t, s] == gp.quicksum(ed[i, t, s] for i in range(I)))

#----------aggregation------------
for t, s in product(range(T), range(S)):
    m2.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s] + gc[t, s] - gd[t, s])
    m2.addConstr(bp[t, s] <= gp.quicksum(R[i, t, s] for i in range(I)))
    m2.addConstr(bp[t, s] <= M2 * mu[t, s])
    m2.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
    m2.addConstr(bm[t, s] <= M2 * eta[t, s])
    m2.addConstr(gc[t, s] <= M2 * (1 - eta[t, s]))
    
m2.optimize()

if m2.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {m2.objVal}")
else:
    print("No optimal solution found.")

Set parameter MIPGap to value 1e-06


Set parameter Heuristics to value 0.3
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  1e-06
Heuristics  0.3

Optimize a model with 47094 rows, 29886 columns and 149886 nonzeros
Model fingerprint: 0xd1b438a7
Variable types: 20526 continuous, 9360 integer (9360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 23146 rows and 14366 columns
Presolve time: 0.14s
Presolved: 23948 rows, 15520 columns, 67782 nonzeros
Variable types: 8697 continuous, 6823 integer (6823 binary)

Root relaxation: objective 7.825744e+05, 10221 iterations, 0.15 seconds (0.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [5]:
a_vals = np.array(
    [m2.getVarByName(f"alpha[{t}]").X for t in range(T)]
)
bp_vals = np.array(
    [[m2.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
bm_vals = np.array(
    [[m2.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
x_vals = np.array(
    [
        [[m2.getVarByName(f"x[{i},{t}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
yp_vals = np.array(
    [
        [
            [m2.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ym_vals = np.array(
    [
        [
            [m2.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zc_vals = np.array(
    [
        [
            [m2.getVarByName(f"z_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zd_vals = np.array(
    [
        [
            [m2.getVarByName(f"z_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
z_vals = np.array(
    [
        [[m2.getVarByName(f"z[{i},{t},{s}]").X for s in range(S)] for t in range(T + 1)]
        for i in range(I)
    ]
)
ep_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
em_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ec_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ed_vals = np.array(
    [
        [
            [m2.getVarByName(f"e_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dp_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dyp[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
dm_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dym[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
dcp_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzc+[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
dcm_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzc-[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
ddp_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzd+[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)
ddm_vals = np.array(
    [
        [
            [
                [m2.getVarByName(f"dzd-[{i},{j},{t},{s}]").X for s in range(S)]
                for t in range(T)
            ]
            for i in range(I)
        ]
        for j in range(I)
    ]
)

In [6]:
for s, t, i in product(range(1,2), range(T), range(I)):
    print(f"[i={i} t={t:02}, s={s:02} BEFORE]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"y⁺={yp_vals[i, t, s]:.2f}, y⁻={ym_vals[i, t, s]:.2f}, "
          f"zᶜ={zc_vals[i, t, s]:.2f}, zᴰ={zd_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")
    
#     print(f"[i={i} t={t:02}, s={s:02} SET(-)]  "
#           f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
#           f"d⁺={dp_vals[i, t, s]:.2f}, d⁻={dm_vals[i, t, s]:.2f}, "
#           f"dᶜ={dcp_vals[i, t, s]:.2f}, dᴰ={ddp_vals[i, t, s]:.2f}, "
#           f"z={z_vals[i, t, s]:.2f}")
    
#     print(f"[i={i} t={t:02}, s={s:02} SET(+)]  "
#           f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
#           f"d⁺= -X-, d⁻= -X-,",
#           f"dᶜ={dcm_vals[i, t, s]:.2f}, dᴰ={ddm_vals[i, t, s]:.2f}, "
#           f"z={z_vals[i, t, s]:.2f}")

    print(f"[i={i} t={t:02}, s={s:02} AFTER ]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"e⁺={ep_vals[i, t, s]:.2f}, e⁻={em_vals[i, t, s]:.2f}, "
          f"eᶜ={ec_vals[i, t, s]:.2f}, eᴰ={ed_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")
    print()

[i=0 t=00, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=-0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=0 t=00, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=1 t=00, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=-0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=1 t=00, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=2 t=00, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=-0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=2 t=00, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=0 t=01, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=0 t=01, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=1 t=01, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=1 t=01, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=2 t=01, s=01 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=2 t=01, s=01 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00